In [1]:
from models import HN, Autoencoder, BASEModel, MODEL
from collections import OrderedDict, defaultdict
from utils import get_default_device, set_seed, f1_loss
from node import Clients

import os
from tqdm import trange
import pandas as pd
import random

import torch
from torch import nn
import torch.utils.data
from torchvision import transforms

import numpy as np


class parameters:
    def __init__(self):
        self.seed = 0
        self.labels_list = ['JOG', 'JUM', 'STD', 'WAL']  # list of activities
        self.outputdim = len(self.labels_list)
        self.data_address = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + "\\Datasets\\MobiNpy_4_Act\\"  # data adress
#         self.data_address = os.path.abspath(os.path.join(
#             os.getcwd(), os.pardir)) + "/Datasets/MobiNpy_4_Act/"  # data adress
        self.trial_number = 1  # which trial we use for this test
        self.label_ratio = 0.10  # ratio of labeled data
        self.eval_ratio = 0.30  # ratio of eval data
        self.number_of_client = 1  # total number of clients
        self.server_ID = [0]  # server ID
        self.batch_size = 128  # training batch size
        self.window_size = 30  # window size (for our case 30)
        self.width = 9  # data dimension (AX, AY, AZ) (GX, GY, GZ) (MX, MY, MZ)
        self.n_kernels = 16  # number of kernels for hypernetwork
        self.total_number_of_clients = 59  # total number of subjects (client + server)
        self.learning_rate = 0.001  # learning rate for optimizer
        self.steps = 10  # total number of epochs
        self.inner_step_for_AE = 5  # number of epochs to fine tunne the Autoencoder
        self.inner_step_server_finetune = 5  # number of steps in the server side to finetune
        self.inner_step_for_model = 5  # number of steps that server fine tune its hn and user embedding parameters
        self.model_loop = False  # feedback loop for user model
        self.inner_step_for_client = 5  # number of steps that user fine tune its model
        self.inner_lr = 0.001  # user learning rate
        self.inner_wd = 5e-5  # weight decay
        self.inout_channels = 1  # number of channels
        self.hidden = 16  # Autoencoder layer 2 parameters
        self.n_kernels_enc = 3  # autoencoder encoder kernel size
        self.hidden_dim_for_HN = 100  # hidden dimension for hypernetwork
        self.n_kernels_dec = 3  # autoencoder decoder kernel size
        self.latent_rep = 4  # latent reperesentation size
        self.n_hidden_HN = 100  # number of hidden layers in hypernetworks
        self.stride_value = 1  # stride value for autoencoder
        self.padding_value = 1  # padding value for autoencoder
        self.model_hidden_layer =128  # final model hidden layer size
        self.spec_norm = False  # True if you want to use spectral norm



In [2]:
params = parameters()
set_seed(params.seed)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
device = get_default_device()



model

In [3]:
model = MODEL(inout_channels=params.inout_channels, hidden=params.hidden, n_kernels_enc=params.n_kernels_enc,
                     n_kernels_dec=params.n_kernels_dec, latent_rep=params.latent_rep, stride_value=params.stride_value,
                     padding_value=params.padding_value, latent_rep_fc= 4 * 3 * 6, out_dim=params.outputdim, hidden_layer=params.model_hidden_layer)  # initializing the autoencoder

In [4]:
model.to(device)

MODEL(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(3, 5), stride=(3, 5), padding=0, dilation=1, ceil_mode=False)
  (batch1): BatchNorm1d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=72, out_features=128, bias=True)
  (batch2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
)

In [5]:
 criteria_model = torch.nn.NLLLoss()

In [6]:
inner_optim = torch.optim.Adam(model.parameters(), lr=params.inner_lr, weight_decay=params.inner_wd)

In [7]:
nodes = Clients(address=params.data_address,
                    trial_number=params.trial_number,
                    label_ratio=params.label_ratio,
                    server_ID=params.server_ID,
                    eval_ratio=params.eval_ratio,
                    window_size=params.window_size,
                    width=params.width,
                    transform=transform,
                    num_user=params.total_number_of_clients)
# dataloaders
client_loader = []
client_labeled_loaders = []
eval_loader = []
server_loaders = torch.utils.data.DataLoader(
    nodes.server_loaders, batch_size=params.batch_size, shuffle=True)

for i in range(params.number_of_client):
    client_loader.append(torch.utils.data.DataLoader(
        nodes.client_loaders[i], batch_size=params.batch_size, shuffle=True))
    client_labeled_loaders.append(torch.utils.data.DataLoader(
        nodes.client_labeled_loaders[i], batch_size=params.batch_size, shuffle=True))
    eval_loader.append(torch.utils.data.DataLoader(
        nodes.eval_data[i], batch_size=params.batch_size, shuffle=True))


In [8]:
model.train()
for i in range(20):
    inner_optim.zero_grad()
    for sensor_values, activity in client_labeled_loaders[0]:
        predicted_activity = model(sensor_values.to(device).float())
        loss = criteria_model(predicted_activity, activity.to(device))
        loss.backward()
        inner_optim.step()
    print('modelfine epoch [{}/{}], loss:{:.4f}'.format(i + 1, params.inner_step_for_model, loss.data))

modelfine epoch [1/5], loss:0.1649
modelfine epoch [2/5], loss:0.2908
modelfine epoch [3/5], loss:0.0979
modelfine epoch [4/5], loss:0.4567
modelfine epoch [5/5], loss:0.1631
modelfine epoch [6/5], loss:0.2107
modelfine epoch [7/5], loss:0.2161
modelfine epoch [8/5], loss:0.0094
modelfine epoch [9/5], loss:0.0112
modelfine epoch [10/5], loss:0.0006
modelfine epoch [11/5], loss:0.0383
modelfine epoch [12/5], loss:0.0002
modelfine epoch [13/5], loss:0.0082
modelfine epoch [14/5], loss:0.1261
modelfine epoch [15/5], loss:0.1513
modelfine epoch [16/5], loss:0.0272
modelfine epoch [17/5], loss:0.0431
modelfine epoch [18/5], loss:0.1837
modelfine epoch [19/5], loss:0.0030
modelfine epoch [20/5], loss:0.0070


In [9]:
with torch.no_grad():
    model.eval()
    prvs_loss_server_model = 0
    f1_score_server = 0
    for sensor_values, activity in eval_loader[0]:
        predicted_activity = model(sensor_values.to(device).float())
        prvs_loss_server_model += criteria_model(predicted_activity, activity.to(device)).item() * sensor_values.size(0)
        f1_score_server += f1_loss(activity, predicted_activity) * sensor_values.size(0)
    prvs_loss_server_model /= len(eval_loader[0].dataset)
    f1_score_server /= len(eval_loader[0].dataset)

print("loss: " + str(prvs_loss_server_model))
print("f1: " + str(f1_score_server))

loss: 0.006253385489769368
f1: 0.998384355370485


Autoencoder

In [26]:
import torch.nn.functional as F
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(270, 256),
            nn.ReLU(True),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.Linear(128, 64))
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.BatchNorm1d(256),
            nn.Linear(256, 270),
            nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [27]:
AE = autoencoder()
AE.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(AE.parameters(), lr=1e-3, weight_decay=1e-5)
AE.train()

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=270, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU(inplace=True)
    (5): Linear(in_features=128, out_features=64, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=64, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU(inplace=True)
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Linear(in_features=256, out_features=270, bias=True)
    (6): Tanh()
  )
)

In [28]:
from torch.autograd import Variable

for epoch in range(100):
    for sensor, _ in client_loader[0]:
        sensor = sensor.view(sensor.size(0), -1)
        sensor = Variable(sensor).to(device)
        # ===================forward=====================
        output = AE(sensor.float())
        loss = criterion(output, sensor.float())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    if epoch % 10 == 0:
        print('AE epoch [{}/{}], loss:{:.4f}'.format(epoch, 100, loss.data))
   
        

AE epoch [0/100], loss:0.5809
AE epoch [10/100], loss:0.4224
AE epoch [20/100], loss:0.3664
AE epoch [30/100], loss:0.2795
AE epoch [40/100], loss:0.2123
AE epoch [50/100], loss:0.2838
AE epoch [60/100], loss:0.2440
AE epoch [70/100], loss:0.3066
AE epoch [80/100], loss:0.2530
AE epoch [90/100], loss:0.2740


In [29]:
import torch.nn.functional as F
class BASEModel(nn.Module):
    def __init__(self): 
        super(BASEModel, self).__init__()

        self.batch1 = nn.BatchNorm1d(64)
        self.batch2 = nn.BatchNorm1d(16)
        self.fc1 = nn.Linear(64, 16)
        self.fc2 = nn.Linear(16, 4)

    def forward(self, x):
        x = self.batch1(x)
        x = self.batch2(F.relu(self.fc1(x)))
        x = F.log_softmax(self.fc2(x), dim=1)
        return x

In [30]:
model = BASEModel()
model.to(device)
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
AE.eval
model.train()

BASEModel(
  (batch1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=64, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=4, bias=True)
)

In [ ]:
for epoch in range(50):
    for sensor, activity in client_labeled_loaders[0]:
        sensor = sensor.view(sensor.size(0), -1)
        encoded = AE.encoder(sensor.to(device).float())
        predicted = model(encoded)
        # ===================forward=====================
        loss = criterion(predicted, activity.to(device))
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    if epoch % 10 == 0:
        print('AE epoch [{}/{}], loss:{:.4f}'.format(epoch, 100, loss.data))


AE epoch [0/100], loss:0.4911
AE epoch [10/100], loss:0.0496


In [16]:
with torch.no_grad():
    model.eval()
    prvs_loss_server_model = 0
    f1_score_server = 0
    for sensor_values, activity in eval_loader[0]:
        sensor_values = sensor_values.view(sensor_values.size(0), -1)
        encoded_sensor = AE.encoder(sensor_values.to(device).float())
        predicted_activity = model(encoded_sensor)
        prvs_loss_server_model += criteria_model(predicted_activity, activity.to(device)).item() * sensor_values.size(0)
        f1_score_server += f1_loss(activity, predicted_activity) * sensor_values.size(0)
    prvs_loss_server_model /= len(eval_loader[0].dataset)
    f1_score_server /= len(eval_loader[0].dataset)

print("loss: " + str(prvs_loss_server_model))
print("f1: " + str(f1_score_server))

loss: 0.002365735809062116
f1: 0.999118982623249


In [17]:
import torch.nn.functional as F
class Autoencoder1(nn.Module):
    def __init__(self, inout_channels=1, hidden=16, n_kernels_enc=3,
                 n_kernels_dec=3, latent_rep=4, stride_value=1, padding_value=1):
        super(Autoencoder1, self).__init__()
        # Encoder
        self.conv1 = nn.Conv2d(inout_channels, hidden,
                               n_kernels_enc, padding=padding_value)  # hidden*9*30
        self.conv2 = nn.Conv2d(
            hidden, latent_rep, n_kernels_enc, padding=padding_value)  # latent_rep*9*30
        self.pool = nn.MaxPool2d((3, 5), stride=(3, 5))  # latent_rep*3*6
        # self.pool = nn.MaxPool2d(3, stride=2)
        # self.conv3 = nn.Conv2d(8, latent_rep, n_kernels_enc, padding=padding_value)
        # batch norms
        self.batchNorm1 = nn.BatchNorm2d(16)
        self.batchNorm2 = nn.BatchNorm2d(4)
        # Decoder

        #        self.t_conv1 = nn.ConvTranspose2d(
        #            latent_rep, hidden, kernel_size=(n_kernels_dec, n_kernels_dec), stride=stride_value, padding=padding_value)
        #        self.t_conv2 = nn.ConvTranspose2d(
        #            hidden, inout_channels, kernel_size=(n_kernels_dec, n_kernels_dec), stride=stride_value,
        #            padding=padding_value)

        self.t_conv1 = nn.ConvTranspose2d(
            latent_rep, hidden, kernel_size=(n_kernels_dec, n_kernels_dec), stride=stride_value, padding=padding_value)
        # self.t_conv2 = nn.ConvTranspose2d(
        #     hidden, latent_rep, kernel_size=(n_kernels_dec, n_kernels_dec), stride=stride_value, padding=padding_value)
        self.t_conv2 = nn.ConvTranspose2d(
            hidden, inout_channels, kernel_size=(3, 5), stride=(3, 5), padding=0)
        print('')

    def encoder(self, x):
        #print("input: " + str(x.shape))
        z = F.relu(self.batchNorm1(self.conv1(x)))
        #print("conv1: " + str(z.shape))
        z = self.pool(F.relu(self.batchNorm2(self.conv2(z))))
        #print("conv2: " + str(z.shape))
        # z = F.relu(self.conv3(z))
        return z

    def decoder(self, x):
        z = F.relu(self.t_conv1(x))
        #print ("t_conv1: " + str(z.shape))
        z = torch.tanh(self.t_conv2(z))
        #print("t_conv2: " + str(z.shape))
        # z = (self.t_conv3(z))
        #print("t_conv3: " + str(z.shape))
        # z = t.sigmoid(self.t_conv3(z))
        return z
    
    def forward(self, input):
        latent_representation = self.encoder(input)
        output = self.decoder(latent_representation)
        return output

In [18]:
AE = Autoencoder1(inout_channels=params.inout_channels, hidden=params.hidden, n_kernels_enc=params.n_kernels_enc,
                     n_kernels_dec=params.n_kernels_dec, latent_rep=params.latent_rep, stride_value=params.stride_value,
                     padding_value=params.padding_value)  # initializing the autoencoder

In [19]:
AE.to(device)

Autoencoder1(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(3, 5), stride=(3, 5), padding=0, dilation=1, ceil_mode=False)
  (batchNorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchNorm2): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (t_conv1): ConvTranspose2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (t_conv2): ConvTranspose2d(16, 1, kernel_size=(3, 5), stride=(3, 5))
)

In [20]:
optimizer = torch.optim.Adam(params=AE.parameters(), lr=0.001)
criteria_AE = torch.nn.MSELoss()

In [21]:
AE.train()
for i in range(20):
    inner_optim.zero_grad()
    for sensor_values, activity in client_loader[0]:
        predicted_activity = AE(sensor_values.to(device).float())
        loss = criteria_AE(predicted_activity.float(), sensor_values.to(device).float())
        loss.backward()
        optimizer.step()
    print('modelfine epoch [{}/{}], loss:{:.4f}'.format(i + 1, params.inner_step_for_model, loss.data))

modelfine epoch [1/5], loss:1.0229
modelfine epoch [2/5], loss:1.0010
modelfine epoch [3/5], loss:0.8084
modelfine epoch [4/5], loss:0.8907
modelfine epoch [5/5], loss:0.7471
modelfine epoch [6/5], loss:0.8624
modelfine epoch [7/5], loss:0.7110
modelfine epoch [8/5], loss:0.8112
modelfine epoch [9/5], loss:0.7762
modelfine epoch [10/5], loss:0.7859
modelfine epoch [11/5], loss:0.5692
modelfine epoch [12/5], loss:0.7571
modelfine epoch [13/5], loss:0.6742
modelfine epoch [14/5], loss:0.7142
modelfine epoch [15/5], loss:0.6274
modelfine epoch [16/5], loss:0.7894
modelfine epoch [17/5], loss:0.6347
modelfine epoch [18/5], loss:0.6619
modelfine epoch [19/5], loss:0.7076
modelfine epoch [20/5], loss:0.7351


In [22]:
with torch.no_grad():
    AE.eval()
    prvs_loss_for_AE = 0
    for sensor_values, _ in eval_loader[0]:
        predicted_sensor_values = AE(sensor_values.to(device).float())
        prvs_loss_for_AE += criteria_AE(predicted_sensor_values.to(device),sensor_values.to(device).float()).item() * sensor_values.size(0)
        # every time, the loss is 47 and the length is around 6680
    prvs_loss_for_AE /= len(eval_loader[0].dataset)
print(prvs_loss_for_AE)

0.6987799470413458


In [23]:
model = BASEModel(latent_rep= 4 * 3 * 6, #params.width * params.window_size * params.latent_rep,
                      out_dim=params.outputdim, hidden_layer=params.model_hidden_layer)  # initilizing the base model

TypeError: __init__() got an unexpected keyword argument 'latent_rep'

In [ ]:
model.to(device)

In [ ]:
inner_optim_base = torch.optim.Adam(model.parameters(), lr=params.inner_lr, weight_decay=params.inner_wd)
for i in range(5):
    inner_optim_base.zero_grad()
    for sensor_values, activity in client_labeled_loaders[0]:
        encoded_sensor_values = AE.encoder(sensor_values.to(device).float())
        predicted_activity = model(encoded_sensor_values)
        loss = criteria_model(predicted_activity, activity.to(device))
        loss.backward()
        inner_optim.step()
    print('modelfine epoch [{}/{}], loss:{:.4f}'.format(i + 1, params.inner_step_for_model, loss.data))

In [ ]:
with torch.no_grad():
    model.eval()
    prvs_loss_server_model = 0
    f1_score_server = 0
    for sensor_values, activity in eval_loader[0]:
        encoded_sensor_values = AE.encoder(sensor_values.to(device).float())
        predicted_activity = model(encoded_sensor_values.to(device).float())
        prvs_loss_server_model += criteria_model(predicted_activity, activity.to(device)).item() * sensor_values.size(0)
        f1_score_server += f1_loss(activity, predicted_activity) * sensor_values.size(0)
    prvs_loss_server_model /= len(eval_loader[0].dataset)
    f1_score_server /= len(eval_loader[0].dataset)

print("loss: " + str(prvs_loss_server_model))
print("f1: " + str(f1_score_server))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(sensor_values[5,0,8,:].cpu())